<a href="https://colab.research.google.com/github/bkaye-ox/TS-AE/blob/master/train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import tsnets
import torch.utils.data as tdata

import plotly.express as px


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")
torch.set_default_tensor_type(torch.cuda.FloatTensor)
#

Using cuda:0 device


In [ ]:
main_settings = dict(
    src='ts_norm.feather',
    nx=8,
    N_y=30,
    N_u=30,
)

In [ ]:

def train(dataloader, model,  loss_params, optimizer, N_epochs, losses):
    size = len(dataloader.dataset)

    # model.set_loss_fn(loss_fn,)

    model.train()
    for epoch in range(N_epochs):
        if int(epoch/N_epochs)/100 % 20 == 0:
            current = epoch
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {losses[-1]:>7f}  [{current:>5d}/{N_epochs:>5d}]")
        for batch, X in enumerate(dataloader):
            X = [x.cuda() for x in X]

            # Compute prediction error
            # pred = model(*X)
            # out = model.get_x_kp1(*X[:2])
            # loss = loss_fn(pred, out)

            loss = model.train_loss(X, loss_params)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

            if batch % 100 == 0:
                # 
                


In [ ]:
train_ds = tsnets.TSDataSet(**main_settings, N_preds=1 test=False)
phase2_ds = tsnets.TSDataSet(**main_settings, N_preds=5,test=False)
test_ds = tsnets.TSDataSet(**main_settings,N_preds=5, test=True)

loader = tdata.DataLoader(
    dataset=train_ds, batch_size=128, shuffle=True, drop_last=True,generator=torch.Generator(device='cuda'),)
test_dataloader = tdata.DataLoader(test_ds, batch_size=128, drop_last=True,generator=torch.Generator(device='cuda'),)

params = {**train_ds.get_params(),  **dict(loss_fn=torch.nn.L1Loss())}
model = tsnets.TSNet(**params).cuda()

phase1_loss = dict(a=1000., b=300, c=0.)
phase2_loss = dict(a=0., b=1000., c=100.)


optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-7)




bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
index div:: -1
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
bad shape
index div:: -1


In [ ]:
losses = []

num_epochs_1 = 30
num_epochs_2 = 60
model.train()
with torch.cuda.device(0):
  train(dataloader=loader, model=model, loss_params=phase1_loss,
        optimizer=optimizer, N_epochs=num_epochs_1, losses=losses)
  train(dataloader=loader, model=model, loss_params=phase2_loss,
        optimizer=optimizer, N_epochs=num_epochs_2, losses=losses)
px.line(y=losses, log_y=True).show()

loss: 821.794312  [    0/ 3267]
loss: 796.291809  [    0/ 3267]
loss: 458.608032  [    0/ 3267]
loss: 272.381866  [    0/ 3267]
loss: 212.229034  [    0/ 3267]
loss: 162.622467  [    0/ 3267]
loss: 202.247467  [    0/ 3267]
loss: 168.308075  [    0/ 3267]
loss: 141.540741  [    0/ 3267]
loss: 146.005722  [    0/ 3267]
loss: 103.485237  [    0/ 3267]
loss: 125.685898  [    0/ 3267]
loss: 132.983780  [    0/ 3267]
loss: 116.234833  [    0/ 3267]
loss: 104.587769  [    0/ 3267]
loss: 101.975677  [    0/ 3267]
loss: 91.632416  [    0/ 3267]
loss: 94.011833  [    0/ 3267]
loss: 68.858414  [    0/ 3267]
loss: 79.599892  [    0/ 3267]
loss: 83.045578  [    0/ 3267]
loss: 85.152122  [    0/ 3267]
loss: 87.105179  [    0/ 3267]
loss: 78.359222  [    0/ 3267]
loss: 62.679970  [    0/ 3267]
loss: 88.602470  [    0/ 3267]
loss: 78.071930  [    0/ 3267]
loss: 68.358376  [    0/ 3267]
loss: 67.832062  [    0/ 3267]
loss: 73.994713  [    0/ 3267]
loss: 151.536896  [    0/ 3267]
loss: 99.045967  [    

In [ ]:

model.eval()

res = []
for I in test_dataloader:
    with torch.no_grad():
        I_km1 = I[0]
        u_k = model._u_k(I_k=I[1])
        preds = model.predict1(I_km1=I_km1, u_k=u_k)
        ground_truth = model._y_k(I_k=I[1])

        res.append([preds.cpu().numpy().reshape(-1,),
                   ground_truth.cpu().numpy().reshape(-1,)])


res = [np.concatenate(l) for l in list(zip(*res))]


df = pd.DataFrame()
df['ypred'], df['y'] = res


In [ ]:
px.line(df, y=['y', 'ypred'])
